In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ucimlrepo import fetch_ucirepo

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
# Get Dataset
wine_quality = fetch_ucirepo(id=186)

# Load data into pd dataframe
df = wine_quality.data.original
df

In [ ]:
X = wine_quality.data.features 
y = wine_quality.data.targets 

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y.values.ravel(),test_size=0.2,random_state=42)

### Decision Trees

### SVM

In [ ]:
from sklearn.svm import SVC

param_grid ={
    'C' : [.1, 1, 10, 100],
    'gamma' : [1, .1, .01, .001],
    'kernel' : ['rbf', 'linear']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5)

grid.fit(X_train, y_train)

print(f"Best Parameters: {grid.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid.best_score_}")

In [ ]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

### Neural Network